In [1]:
"""
created on 10th Mar 2019

@author: Huan Zheng
"""
# 不用Sequential模型的解决方案：Keras函数式API

'\ncreated on 10th Mar 2019\n\n@author: Huan Zheng\n'

# 函数式API简介

In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32,))

dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

In [5]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='relu'))

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((10000, 64))
y_train = np.random.random((10000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)

Epoch 1/10
10000/10000 [==============================] - 5s 460us/step - loss: 11.5585
Epoch 2/10
10000/10000 [==============================] - 1s 85us/step - loss: 11.5389
Epoch 3/10
10000/10000 [==============================] - 1s 84us/step - loss: 11.5369
Epoch 4/10
10000/10000 [==============================] - 1s 84us/step - loss: 11.5361
Epoch 5/10
10000/10000 [==============================] - 1s 86us/step - loss: 11.5355
Epoch 6/10
10000/10000 [==============================] - 1s 88us/step - loss: 11.5350
Epoch 7/10
10000/10000 [==============================] - 1s 87us/step - loss: 11.5347
Epoch 8/10
10000/10000 [==============================] - 1s 87us/step - loss: 11.5342
Epoch 9/10
10000/10000 [==============================] - 1s 87us/step - loss: 11.5337
Epoch 10/10
10000/10000 [==============================] - 1s 93us/step


In [8]:
print(score)

11.532308128356934


# 多输入模型

## 用函数式API实现双输入问答模型

In [13]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')

embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)

encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

## 将数据输入到多输入模型中

In [22]:
import numpy as np
import keras

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(1, answer_vocabulary_size, size=(num_samples))

answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 11s 11ms/step - loss: 6.2150 - acc: 0.0060
Epoch 2/10
1000/1000 [==============================] - 9s 9ms/step - loss: 6.2014 - acc: 0.0440
Epoch 3/10
1000/1000 [==============================] - 9s 9ms/step - loss: 6.1799 - acc: 0.0500
Epoch 4/10
1000/1000 [==============================] - 9s 9ms/step - loss: 6.1099 - acc: 0.0100
Epoch 5/10
1000/1000 [==============================] - 9s 9ms/step - loss: 6.0518 - acc: 0.0050
Epoch 6/10
1000/1000 [==============================] - 10s 10ms/step - loss: 5.9819 - acc: 0.0080
Epoch 7/10
1000/1000 [==============================] - 9s 9ms/step - loss: 5.8911 - acc: 0.0090
Epoch 8/10
1000/1000 [==============================] - 9s 9ms/step - loss: 5.7921 - acc: 0.0100
Epoch 9/10
1000/1000 [==============================] - 9s 9ms/step - loss: 5.7129 - acc: 0.0430
Epoch 10/10
1000/1000 [==============================] - 9s 9ms/step - loss: 5.6295 - acc: 0.0540


# 多输出模型

## 用函数API实现一个三输出模型

In [25]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

## 多输出模型编译选项：多重损失

In [27]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop', loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'})

## 多输出模型编译选项：损失加权

In [29]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights = [0.25, 1., 10.])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'},
              loss_weights = {'age': 0.25, 'income': 1., 'gender': 10.})

## 将数据输入到多输出模型当中

In [ ]:
model.fit(posts, [age_target, income_target, gender_target], epochs=100, batch_size=64)

model.fit(posts, {'age': age_target, 'income': income_target, 'gender': gender_target}, epochs=100, batch_size=64)

# 层组成的有向无环图
## Inception模块

## 残差连接